# Exploring Agentic Patterns with OpenAI API

This notebook explores common workflow and agentic design patterns for building AI applications, based on the concepts presented in Phil Schmid's blog post "Zero to One: Learning Agentic Patterns". We will demonstrate how to implement these patterns using the OpenAI API.

Agentic systems are characterized by their ability to dynamically plan and execute tasks, often leveraging external tools and memory to achieve complex goals. Understanding these patterns provides a blueprint for designing scalable, modular, and adaptable AI systems.

**Patterns Covered:**

*   **Workflow Patterns:**
    *   Prompt Chaining
    *   Routing or Handoff
    *   Parallelization
*   **Agentic Patterns:**
    *   Reflection
    *   Tool Use (Function Calling)
    *   Planning (Orchestrator-Workers)
    *   Multi-Agent

## 1. Setup

First, you need to install the OpenAI Python client library and set up your API key.

You can install the library using pip:

```bash
pip install openai

In [5]:
import os
from openai import OpenAI

# Set your OpenAI API key
# It's recommended to set this as an environment variable (OPENAI_API_KEY)
# If you must set it here, replace 'YOUR_OPENAI_API_KEY' with your actual key
# os.environ["OPENAI_API_KEY"] = "YOUR_OPENAI_API_KEY"

# Initialize the OpenAI client
# The client automatically reads the OPENAI_API_KEY environment variable
client = OpenAI()

print("OpenAI client initialized.")

OpenAI client initialized.


## 2. OpenAI API Basics
The core of interacting with OpenAI models for these patterns is typically the Chat Completions API. This API allows you to have multi-turn conversations and is versatile enough to handle various tasks, including generating text, following instructions, and using tools.
The basic structure involves sending a list of messages with different roles (system, user, assistant) to the API.

In [ ]:
def get_completion(messages, model="gpt-4.1-nano", temperature=0):
    """Helper function to get completion from OpenAI chat model."""
    try:
        response = client.chat.completions.create(
            model=model,
            messages=messages,
            temperature=temperature,
        )
        print(response)
        return response.choices[0].message.content
    except Exception as e:
        print(f"An error occurred: {e}")
        return None

# Example basic call
messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "What is the capital of France?"}
]

response = get_completion(messages)
print(response)

ChatCompletion(id='chatcmpl-Bcqf5hjxtOqHxCEqJWVwZc6vFBLcW', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='The capital of France is Paris.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1748598595, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_07871e2ad8', usage=CompletionUsage(completion_tokens=7, prompt_tokens=24, total_tokens=31, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
The capital of France is Paris.


In [9]:
response.choices[0].message.content

'The capital of France is Paris.'

## 3. Workflow Patterns
Workflow patterns involve predefined sequences or routing logic.
### 3.1. Prompt Chaining
The output of one LLM call feeds sequentially into the input of the next. This decomposes a task into a fixed sequence of steps.
Use Case: Summarize a text, then translate the summary.

In [11]:
# --- Step 1: Summarize Text ---
original_text = "Large language models are powerful AI systems trained on vast amounts of text data. They can generate human-like text, translate languages, write different kinds of creative content, and answer your questions in an informative way."

messages_summary = [
    {"role": "system", "content": "You are a helpful assistant that summarizes text."},
    {"role": "user", "content": f"Summarize the following text in one sentence: {original_text}"}
]

summary = get_completion(messages_summary)
print(f"Summary: {summary}")

# --- Step 2: Translate the Summary ---
if summary:
    messages_translate = [
        {"role": "system", "content": "You are a helpful assistant that translates text into French."},
        {"role": "user", "content": f"Translate the following summary into French, only return the translation, no other text: {summary}"}
    ]

    translation = get_completion(messages_translate)
    print(f"Translation: {translation}")

ChatCompletion(id='chatcmpl-BcqfpWgeYs7O1GQTGycK8r6FuL0Va', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='Large language models are advanced AI systems capable of generating human-like text, translating languages, creating various types of content, and providing informative answers.', refusal=None, role='assistant', annotations=[], audio=None, function_call=None, tool_calls=None))], created=1748598641, model='gpt-4o-2024-08-06', object='chat.completion', service_tier='default', system_fingerprint='fp_90122d973c', usage=CompletionUsage(completion_tokens=28, prompt_tokens=71, total_tokens=99, completion_tokens_details=CompletionTokensDetails(accepted_prediction_tokens=0, audio_tokens=0, reasoning_tokens=0, rejected_prediction_tokens=0), prompt_tokens_details=PromptTokensDetails(audio_tokens=0, cached_tokens=0)))
Summary: Large language models are advanced AI systems capable of generating human-like text, translating languages, creati

### 3.2. Routing or Handoff
An initial LLM acts as a router, classifying the user's input and directing it to the most appropriate specialized task or LLM (or prompt).
Use Case: Route a user query to a specific category (e.g., weather, science, general).
We can use function calling or structured output via the prompt to achieve routing. Let's use a prompt-based approach for simplicity here, asking the model to output a specific format.

In [14]:
import json
from pydantic import BaseModel
from enum import Enum

# Define routing schemas
class Category(Enum):
    WEATHER = "weather"
    SCIENCE = "science"
    CODING = "coding"
    UNKNOWN = "unknown"

class RoutingDecision(BaseModel):
    category: Category
    reasoning: str
    confidence: float

def route_query(user_query: str) -> RoutingDecision:
    """Route user query to appropriate category"""
    
    prompt_router = f"""
    Analyze the user query and determine its category.
    
    Categories:
    - weather: Questions about weather conditions, forecasts, climate
    - science: Questions about scientific concepts, theories, research
    - coding: Questions about programming, software development, technical issues
    - unknown: If the category is unclear or doesn't fit above categories
    
    Query: {user_query}
    
    Respond with JSON containing:
    - category: one of the categories above
    - reasoning: brief explanation for the categorization
    - confidence: float between 0.0 and 1.0 indicating confidence level
    """
    
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[{"role": "user", "content": prompt_router}],
        response_format={"type": "json_object"},
        temperature=0.2
    )
    
    result = json.loads(response.choices[0].message.content)
    # Validate with Pydantic model
    try:
        return RoutingDecision(**result)
    except Exception as e:
        print(f"Warning: Could not parse routing decision JSON: {result}. Error: {e}")
        # Return a default unknown category if parsing fails
        return RoutingDecision(category=Category.UNKNOWN, reasoning=f"Parsing failed: {e}. Original JSON: {result}", confidence=0.0)

def handle_weather_query(query: str) -> str:
    """Handle weather-related queries"""
    prompt = f"""
    You are a weather specialist. Provide a helpful response about weather.
    If specific location data is needed but not available, provide general weather advice.
    
    Query: {query}
    """
    
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.3
    )
    
    return response.choices[0].message.content

def handle_science_query(query: str) -> str:
    """Handle science-related queries"""
    prompt = f"""
    You are a science expert. Provide a clear, accurate explanation.
    Use analogies when helpful and break down complex concepts.
    
    Query: {query}
    """
    
    response = client.chat.completions.create(
        model="gpt-4o",  # Use more capable model for science
        messages=[{"role": "user", "content": prompt}],
        temperature=0.2
    )
    
    return response.choices[0].message.content

def handle_coding_query(query: str) -> str:
    """Handle coding-related queries"""
    prompt = f"""
    You are a programming expert. Provide clear, practical coding help.
    Include code examples when appropriate and explain your reasoning.
    
    Query: {query}
    """
    
    response = client.chat.completions.create(
        model="gpt-4o",  # Use more capable model for coding
        messages=[{"role": "user", "content": prompt}],
        temperature=0.1
    )
    
    return response.choices[0].message.content

def handle_unknown_query(query: str, reasoning: str) -> str:
    """Handle queries that don't fit specific categories"""
    prompt = f"""
    The user query couldn't be categorized into weather, science, or coding.
    Reasoning: {reasoning}
    
    Please provide a helpful general response and suggest how the user might rephrase 
    their question for better assistance.
    
    Query: {query}
    """
    
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=[{"role": "user", "content": prompt}],
        temperature=0.4
    )
    
    return response.choices[0].message.content

def routing_workflow(user_query: str) -> str:
    """Complete routing workflow"""
    print("=== ROUTING WORKFLOW EXAMPLE ===")
    print(f"User Query: {user_query}\n")
    
    # Step 1: Route the query
    routing_decision = route_query(user_query)
    print(f"Routing Decision:")
    print(f"  Category: {routing_decision.category.value}")
    print(f"  Confidence: {routing_decision.confidence:.2f}")
    print(f"  Reasoning: {routing_decision.reasoning}\n")
    
    # Step 2: Handle based on routing
    if routing_decision.category == Category.WEATHER:
        final_response = handle_weather_query(user_query)
    elif routing_decision.category == Category.SCIENCE:
        final_response = handle_science_query(user_query)
    elif routing_decision.category == Category.CODING:
        final_response = handle_coding_query(user_query)
    else:
        final_response = handle_unknown_query(user_query, routing_decision.reasoning)
    
    print(f"Final Response:\n{final_response}")
    return final_response

# Test different types of queries
test_queries = [
    "What's the weather like in Paris today?",
    "What's the best pizza topping?"
]

for query in test_queries:
    routing_workflow(query)
    print("\n" + "="*60 + "\n")

=== ROUTING WORKFLOW EXAMPLE ===
User Query: What's the weather like in Paris today?

Routing Decision:
  Category: weather
  Confidence: 0.95
  Reasoning: The user is asking about current weather conditions in Paris, which clearly relates to weather information.

Final Response:
I'm glad you're interested in the weather in Paris! While I don't have real-time data, I can offer some general advice. Typically, spring in Paris brings mild temperatures with a mix of sunshine and occasional showers. It's a good idea to carry an umbrella and dress in layers to stay comfortable. For the most accurate and up-to-date weather forecast, I recommend checking a reliable weather service or app before your plans.


=== ROUTING WORKFLOW EXAMPLE ===
User Query: What's the best pizza topping?

Routing Decision:
  Category: unknown
  Confidence: 0.90
  Reasoning: The query asks about pizza toppings, which is related to food and culinary preferences, not fitting into weather, science, or coding categories

### 3.3. Parallelization

A task is broken down into independent subtasks that are processed simultaneously by multiple LLMs, with their outputs being aggregated.

**Use Cases:**
- RAG with query decomposition
- Document analysis (parallel section processing)
- Generating multiple perspectives
- Map-reduce operations

In [15]:
import asyncio
import time
from openai import AsyncOpenAI

# Initialize the Async OpenAI client
async_client = AsyncOpenAI()

async def generate_content_async(prompt: str, model: str = "gpt-4.1-nano", temperature: float = 0.7) -> str:
    """Generate content asynchronously"""
    response = await async_client.chat.completions.create(
        model=model,
        messages=[{"role": "user", "content": prompt}],
        temperature=temperature
    )
    return response.choices[0].message.content.strip()

async def parallel_story_generation():
    """Generate multiple story perspectives in parallel"""
    print("=== PARALLELIZATION EXAMPLE ===")
    
    topic = "a friendly robot exploring a jungle"
    print(f"Topic: {topic}\n")
    
    # Define parallel tasks with different perspectives
    prompts = [
        f"Write a short, adventurous story idea (2-3 sentences) about {topic}. Focus on excitement and discovery.",
        f"Write a short, funny story idea (2-3 sentences) about {topic}. Focus on humor and mishaps.",
        f"Write a short, heartwarming story idea (2-3 sentences) about {topic}. Focus on friendship and connection."
    ]
    
    # Run tasks concurrently
    start_time = time.time()
    tasks = [generate_content_async(prompt) for prompt in prompts]
    results = await asyncio.gather(*tasks)
    end_time = time.time()
    
    print(f"Parallel execution time: {end_time - start_time:.2f} seconds\n")
    
    # Display individual results
    story_types = ["Adventurous", "Funny", "Heartwarming"]
    print("--- Individual Story Ideas ---")
    for i, (story_type, result) in enumerate(zip(story_types, results)):
        print(f"{story_type}: {result}\n")
    
    # Aggregate results
    story_ideas = '\n'.join([f"{story_type}: {result}" for story_type, result in zip(story_types, results)])
    
    aggregation_prompt = f"""
    Combine the following 3 story perspectives into a single, cohesive story outline 
    that incorporates elements from each perspective:
    
    {story_ideas}
    
    Create a unified story that balances adventure, humor, and heart.
    """
    
    aggregated_response = await generate_content_async(
        aggregation_prompt, 
        model="gpt-4.1-nano", 
        temperature=0.6
    )
    
    print("--- Aggregated Story Outline ---")
    print(aggregated_response)
    
    return {
        "individual_stories": dict(zip(story_types, results)),
        "aggregated_story": aggregated_response,
        "execution_time": end_time - start_time
    }

# Run the parallel example
# In a notebook, you can run async functions directly
parallel_result = await parallel_story_generation()

=== PARALLELIZATION EXAMPLE ===
Topic: a friendly robot exploring a jungle

Parallel execution time: 2.56 seconds

--- Individual Story Ideas ---
Adventurous: In the heart of an uncharted jungle, a curious robot named Riko ventures beyond its usual circuits, eager to uncover ancient hidden temples and shimmering waterfalls. As it navigates tangled vines and whispers of forgotten secrets, Riko’s sensors pick up mysterious signals, hinting at a legendary artifact that could change everything. With every step, the robot’s excitement grows, driven by the thrill of discovery and the promise of adventure.

Funny: Robot Rumble, eager to explore the jungle, tried to make friends with a curious parrot—only to be greeted with a squawk so loud it triggered his hiccup function. Suddenly, he was bouncing around like a pogo stick, chasing his own tail because he thought it was a jungle vine. Turns out, even friendly robots can get tangled up in their own circuits when faced with jungle chaos!

Heart

## 4. Agentic Patterns
Agentic patterns involve more dynamic decision-making, self-correction, and interaction with tools or other agents.
### 4.1. Reflection
An agent evaluates its own output and uses that feedback to refine its response iteratively.
Use Case: Write a poem and refine it based on critique.

In [17]:
def generate_poem(topic: str, feedback: str = None) -> str:
    messages = [
        {"role": "system", "content": "You are a creative poet. Write a short, two-line poem."},
        {"role": "user", "content": f"Write a short, two-line poem about {topic}."}
    ]
    if feedback:
        messages.append({"role": "user", "content": f"Please revise the previous poem based on this feedback: {feedback}"})

    poem = get_completion(messages, model="gpt-4.1-nano", temperature=0.8)
    print(f"Generated Poem:\n{poem}")
    return poem

def evaluate_poem(poem: str) -> dict:
    print("\n--- Evaluating Poem ---")
    messages = [
        {"role": "system", "content": """Critique the following poem.
        Does it rhyme well? Is it exactly four lines? Is it creative?
        Respond with a JSON object containing 'status' ('PASS' or 'FAIL') and 'feedback' (string).
        Example: {"status": "FAIL", "feedback": "The poem does not rhyme."}
        """},
        {"role": "user", "content": f"Poem:\n{poem}"}
    ]

    evaluation_text = get_completion(messages, model="gpt-3.5-turbo", temperature=0)

    try:
        evaluation = json.loads(evaluation_text)
        print(f"Evaluation Status: {evaluation.get('status')}")
        print(f"Evaluation Feedback: {evaluation.get('feedback')}")
        return evaluation
    except json.JSONDecodeError:
        print("Failed to parse evaluation.")
        return {"status": "FAIL", "feedback": "Could not parse evaluation response."}

# Reflection Loop
max_iterations = 3
current_iteration = 0
topic = "a robot learning to paint"
current_poem = generate_poem(topic) # Initial generation

while current_iteration < max_iterations:
    current_iteration += 1
    print(f"\n--- Iteration {current_iteration} ---")

    evaluation_result = evaluate_poem(current_poem)

    if evaluation_result.get("status") == "PASS":
        print("\nEvaluation Passed. Final Poem:")
        print(current_poem)
        break
    else:
        feedback = evaluation_result.get("feedback", "")
        print(f"Refining poem based on feedback: {feedback}")
        current_poem = generate_poem(topic, feedback=feedback)

    if current_iteration == max_iterations:
        print("\nMax iterations reached. Last attempt:")
        print(current_poem)

Generated Poem:
Metal fingers hesitate, then glide,  
Colors bloom where circuits once lied.

--- Iteration 1 ---

--- Evaluating Poem ---
Evaluation Status: FAIL
Evaluation Feedback: The poem is only two lines long, not four.
Refining poem based on feedback: The poem is only two lines long, not four.
Generated Poem:
Metal hands brush colors anew,  
Dreams in circuits come into view.

--- Iteration 2 ---

--- Evaluating Poem ---
Evaluation Status: FAIL
Evaluation Feedback: The poem is not exactly four lines.
Refining poem based on feedback: The poem is not exactly four lines.
Generated Poem:
In circuits' glow, a robot's brush takes flight,  
Learning colors' dance beneath the dawn's first light.

--- Iteration 3 ---

--- Evaluating Poem ---
Evaluation Status: FAIL
Evaluation Feedback: The poem is not exactly four lines.
Refining poem based on feedback: The poem is not exactly four lines.
Generated Poem:
In circuits' hum, a brush takes flight,  
A robot's dream in colors bright.

Max it

### 4.2. Tool Use Pattern (Function Calling)

LLM has the ability to invoke external functions or APIs to interact with the outside world.

**Use Cases:**
- API integrations
- Database queries
- External service interactions
- Real-time data retrieval

In [21]:
# Define tool functions
def get_current_temperature(location: str):
    """Get the current temperature in a given location."""
    # In a real implementation, this would call a weather API
    import random
    
    # Simulate data for a few locations
    temperatures = {
        "london": {"temperature": 15, "unit": "celsius", "condition": "Cloudy"},
        "paris": {"temperature": 18, "unit": "celsius", "condition": "Sunny"},
        "new york": {"temperature": 22, "unit": "fahrenheit", "condition": "Partly cloudy"},
        "tokyo": {"temperature": 25, "unit": "celsius", "condition": "Clear"},
        "sydney": {"temperature": 28, "unit": "celsius", "condition": "Rainy"},
    }
    
    location_key = location.lower()
    if location_key in temperatures:
        return temperatures[location_key]
    else:
        # Return a default or random temperature for unknown locations
        return {
            "temperature": random.randint(5, 35),
            "unit": random.choice(["celsius", "fahrenheit"]),
            "condition": random.choice(["Sunny", "Cloudy", "Rainy", "Snowy"])
        }

def get_n_day_weather_forecast(location: str, num_days: int):
    """Get the N-day weather forecast for a given location."""
     # In a real implementation, this would call a weather forecast API
    import random

    forecasts = [
        {"date": "today", "temperature": random.randint(10, 20), "unit": "celsius", "condition": "Cloudy"},
        {"date": "tomorrow", "temperature": random.randint(15, 25), "unit": "celsius", "condition": "Sunny"},
        {"date": "day after tomorrow", "temperature": random.randint(12, 22), "unit": "celsius", "condition": "Rainy"},
    ]
    
    return {"location": location, "num_days": num_days, "forecast": forecasts[:num_days]}

# Map tool names to functions
available_tools = {
    "get_current_temperature": get_current_temperature,
    "get_n_day_weather_forecast": get_n_day_weather_forecast,
}

# Define tool specifications for the API
tools_spec = [
    {
        "type": "function",
        "function": {
            "name": "get_current_temperature",
            "description": "Get the current temperature in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                },
                "required": ["location"],
            },
        },
    },
     {
        "type": "function",
        "function": {
            "name": "get_n_day_weather_forecast",
            "description": "Get the N-day weather forecast for a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                     "num_days": {
                        "type": "integer",
                        "description": "The number of days to forecast",
                    },
                },
                "required": ["location", "num_days"],
            },
        },
    },
]

def tool_use_example(user_query: str):
    """Demonstrate tool use (function calling) with OpenAI"""
    print("=== TOOL USE EXAMPLE ===")
    print(f"User Query: {user_query}\n")
    
    messages = [{"role": "user", "content": user_query}]
    
    # Step 1: Send user query and tool specs to the model
    print("Calling model to decide on tools...")
    response = client.chat.completions.create(
        model="gpt-4.1-nano",
        messages=messages,
        tools=tools_spec,
        tool_choice="auto",  # auto is default, but we'll be explicit
    )
    print("Model responded.\n")
    
    response_message = response.choices[0].message
    tool_calls = response_message.tool_calls
    
    # Step 2: Check if the model wanted to call a tool
    if tool_calls:
        print(f"Model requested tool calls: {len(tool_calls)}")
        # Add the model's tool_calls to the conversation history
        messages.append(response_message)
        
        # Step 3: Call the tools
        for tool_call in tool_calls:
            function_name = tool_call.function.name
            function_to_call = available_tools.get(function_name)
            
            if function_to_call:
                function_args = json.loads(tool_call.function.arguments)
                
                print(f"\nCalling function: {function_name} with args: {function_args}")
                try:
                    function_response = function_to_call(**function_args)
                    print(f"Function call successful. Response: {function_response}")
                    
                    # Add tool response to conversation history
                    messages.append({
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": json.dumps(function_response),
                    })
                except Exception as e:
                    print(f"Error calling function {function_name}: {e}")
                     # Add error response to conversation history
                    messages.append({
                        "tool_call_id": tool_call.id,
                        "role": "tool",
                        "name": function_name,
                        "content": json.dumps({"error": str(e)}),
                    })
            else:
                 print(f"\nError: Function {function_name} not found in available_tools.")
                 messages.append({
                    "tool_call_id": tool_call.id,
                    "role": "tool",
                    "name": function_name,
                    "content": json.dumps({"error": f"Tool '{function_name}' not available."}),
                })
        
        # Step 4: Send the conversation back to the model with tool output
        print("\nSending tool results back to model for final response...")
        final_response = client.chat.completions.create(
            model="gpt-4.1-nano",
            messages=messages,
        )
        print("Model provided final response.")
        print("--- Final Response ---")
        print(final_response.choices[0].message.content)
        return final_response.choices[0].message.content
        
    else:
        # If no tool calls were requested, the model's initial response is the final one
        print("Model did not request tool calls.")
        print("--- Final Response ---")
        print(response_message.content)
        return response_message.content

# Test the tool use pattern
queries_with_tools = [
    "What is the current temperature in London?",
    "What is the 3-day weather forecast for New York?",
    "Tell me a fun fact.", # Should not trigger a tool call
]

for query in queries_with_tools:
    tool_use_example(query)
    print("\n" + "="*60 + "\n")

=== TOOL USE EXAMPLE ===
User Query: What is the current temperature in London?

Calling model to decide on tools...
Model responded.

Model requested tool calls: 1

Calling function: get_current_temperature with args: {'location': 'London'}
Function call successful. Response: {'temperature': 15, 'unit': 'celsius', 'condition': 'Cloudy'}

Sending tool results back to model for final response...
Model provided final response.
--- Final Response ---
The current temperature in London is 15°C and the weather is cloudy.


=== TOOL USE EXAMPLE ===
User Query: What is the 3-day weather forecast for New York?

Calling model to decide on tools...
Model responded.

Model requested tool calls: 1

Calling function: get_n_day_weather_forecast with args: {'location': 'New York', 'num_days': 3}
Function call successful. Response: {'location': 'New York', 'num_days': 3, 'forecast': [{'date': 'today', 'temperature': 16, 'unit': 'celsius', 'condition': 'Cloudy'}, {'date': 'tomorrow', 'temperature': 24, 

### 4.3. Planning Pattern (Orchestrator-Workers)

A dedicated 'orchestrator' LLM plans a complex task by breaking it into smaller subtasks. These subtasks are then executed by specialized 'worker' LLMs or other agents.

**Use Cases:**
- Complex research tasks
- Multi-step content generation
- Coordinating multiple agents/tools
- Executing workflows requiring different expertise

In [22]:
from typing import List, Optional

class Task(BaseModel):
    task_id: int
    description: str
    assignee: str # e.g., "researcher", "writer", "summarizer"
    status: str = "pending"
    result: Optional[str] = None

class Plan(BaseModel):
    overall_goal: str
    tasks: List[Task]

def orchestrator_plan(goal: str) -> Plan:
    """Orchestrator LLM creates a plan of tasks."""
    prompt = f"""
    You are an expert project planner. Break down the following overall goal into a sequence of distinct tasks.
    Assign each task to a suitable worker: 'researcher', 'writer', or 'summarizer'.
    
    Overall Goal: {goal}
    
    Provide the plan as a JSON object with the following structure:
    {{ "overall_goal": "...", "tasks": [ {{ "task_id": int, "description": "...", "assignee": "researcher"|"writer"|"summarizer" }} , ... ] }}
    
    Ensure task IDs are sequential starting from 1.
    """
    
    print("Orchestrator: Creating plan...")
    response = client.chat.completions.create(
        model="gpt-4.1-nano", # Use a capable model for planning
        messages=[{"role": "system", "content": "You are a helpful planning assistant that outputs valid JSON."},
                  {"role": "user", "content": prompt}],
        response_format={"type": "json_object"},
        temperature=0.1
    )
    
    plan_data = json.loads(response.choices[0].message.content)
    
    # Validate with Pydantic model
    try:
        plan = Plan(**plan_data)
        print("Orchestrator: Plan created successfully.")
        for task in plan.tasks:
             print(f"  Task {task.task_id} [{task.assignee}]: {task.description}")
        return plan
    except Exception as e:
        print(f"Error parsing orchestrator plan JSON: {e}. Original JSON: {plan_data}")
        # Return a minimal plan indicating failure
        return Plan(overall_goal=goal, tasks=[Task(task_id=1, description=f"Failed to create plan: {e}", assignee="unknown", status="failed")])
    

def worker_execute_task(task: Task) -> str:
    """Worker LLM executes a specific task."""
    print(f"  Worker [{task.assignee}] executing task {task.task_id}: {task.description}")
    
    # Define prompts based on assignee type
    if task.assignee == "researcher":
        prompt = f"Research and provide key information or facts related to the following request: {task.description}"
        model = "gpt-4.1-nano" # Can use a cheaper model for simple info retrieval
        temp = 0.3
    elif task.assignee == "writer":
        prompt = f"Write a paragraph or section based on the following instruction/topic: {task.description}"
        model = "gpt-4.1-nano" # Use a capable model for writing
        temp = 0.7
    elif task.assignee == "summarizer":
         prompt = f"Summarize the following content based on this instruction: {task.description}"
         model = "gpt-4.1-nano"
         temp = 0.2
    else:
        return f"Error: Unknown assignee type: {task.assignee}"
        
    try:
        response = client.chat.completions.create(
            model=model,
            messages=[{"role": "user", "content": prompt}],
            temperature=temp
        )
        result = response.choices[0].message.content.strip()
        print(f"  Worker [{task.assignee}] task {task.task_id} finished.")
        return result
    except Exception as e:
        print(f"  Worker [{task.assignee}] task {task.task_id} failed: {e}")
        return f"Task failed: {e}"

def planning_workflow(goal: str) -> List[Task]:
    """Complete planning and execution workflow"""
    print("=== PLANNING PATTERN EXAMPLE ===")
    print(f"Overall Goal: {goal}\n")
    
    # Step 1: Orchestrator creates the plan
    plan = orchestrator_plan(goal)
    
    if not plan.tasks or plan.tasks[0].status == "failed":
        print("Planning failed. Aborting.")
        return plan.tasks

    print("\nExecuting Plan...")
    
    # Step 2: Execute tasks sequentially by workers (could be parallelized if tasks are independent)
    executed_tasks = []
    for task in plan.tasks:
        task.status = "in_progress"
        task.result = worker_execute_task(task)
        task.status = "completed" if not task.result.startswith("Task failed:") else "failed"
        executed_tasks.append(task)
        print(f"  Task {task.task_id} Status: {task.status}\n")
        
        # In a real system, results might be passed to subsequent tasks or aggregated
        # For this example, we just print the result of each task
        print(f"  Task {task.task_id} Result Snippet: {task.result[:150]}...\n")
    
    print("Plan Execution Finished.")
    
    # Step 3: (Optional) Orchestrator could aggregate or refine final output
    # For simplicity, we just return the executed tasks
    
    return executed_tasks

# Test the planning workflow
planning_result = planning_workflow("Write a brief report about the benefits of renewable energy, including some key statistics and a concluding summary.")

print("\n=== FINAL EXECUTED TASKS ===")
for task in planning_result:
    print(f"Task {task.task_id} [{task.assignee}] ({task.status}): {task.description}")
    # print(f"  Result: {task.result[:100]}...") # Print result snippet
    print("-"*20)

# Example of aggregating results (Manual for this simple case)
final_report_sections = [task.result for task in planning_result if task.status == "completed"]
print("\n=== AGGREGATED DRAFT REPORT ===")
print("\n".join(final_report_sections))
print("="*60)

=== PLANNING PATTERN EXAMPLE ===
Overall Goal: Write a brief report about the benefits of renewable energy, including some key statistics and a concluding summary.

Orchestrator: Creating plan...
Orchestrator: Plan created successfully.
  Task 1 [researcher]: Conduct research to gather key statistics and information on the benefits of renewable energy.
  Task 2 [writer]: Draft the report by organizing the researched information into a coherent structure.
  Task 3 [summarizer]: Summarize the main points and write a concluding summary for the report.

Executing Plan...
  Worker [researcher] executing task 1: Conduct research to gather key statistics and information on the benefits of renewable energy.
  Worker [researcher] task 1 finished.
  Task 1 Status: completed

  Task 1 Result Snippet: Certainly! Here are key statistics and facts highlighting the benefits of renewable energy:

1. **Environmental Benefits:**
   - **Reduces Greenhouse ...

  Worker [writer] executing task 2: Draft th

### 4.4. Multi-Agent
Multiple distinct agents, each with a specific role or expertise, collaborate to achieve a common goal. This can involve a coordinator agent or handoff logic.
Use Case: Simulate a simple handoff between a Hotel Agent and a Restaurant Agent based on the user's request.
We can simulate this by defining system prompts for each agent and using routing/handoff logic (similar to the Routing pattern, but with distinct agent personas).

In [23]:
from typing import List, Optional, Dict

class Agent:
    def __init__(self, name: str, role: str, description: str, model: str = "gpt-4o-mini"):
        self.name = name
        self.role = role
        self.description = description
        self.model = model
        self.conversation_history = [{
            "role": "system", 
            "content": f"You are {self.name}, a {self.role}. Your goal is to {self.description}. Stay in character and contribute to the discussion."
        }]
    
    def send_message(self, recipient_name: str, content: str):
        """Simulate sending a message to another agent."""
        message = {"sender": self.name, "recipient": recipient_name, "content": content}
        # In a real system, this would go into a shared message queue or similar
        # For this simulation, we just return the message
        return message
        
    def receive_message(self, message: Dict[str, str]):
        """Integrate a received message into conversation history."""
        # Add message from other agent to history as 'user' role from recipient's perspective
        self.conversation_history.append({"role": "user", "content": f"Message from {message['sender']}: {message['content']}"})
        print(f"  {self.name} received message from {message['sender']}.")
        
    def generate_response(self, query: str = None) -> str:
        """Generate a response based on current history and optional query."""
        messages_for_api = list(self.conversation_history) # Copy history
        if query:
            messages_for_api.append({"role": "user", "content": query})
            
        print(f"  {self.name} is generating response...")
        try:
            response = client.chat.completions.create(
                model=self.model,
                messages=messages_for_api,
                temperature=0.7 # Allow creativity in interaction
            )
            content = response.choices[0].message.content.strip()
            
            # Add the generated response to the agent's history
            self.conversation_history.append({"role": "assistant", "content": content})
            print(f"  {self.name} generated response.")
            return content
        except Exception as e:
            print(f"  {self.name} failed to generate response: {e}")
            return f"[Agent {self.name} Error: {e}]"

def multi_agent_debate(topic: str, num_turns: int = 3):
    """Simulate a multi-agent debate."""
    print("=== MULTI-AGENT PATTERN EXAMPLE ===")
    print(f"Debate Topic: {topic}")
    print(f"Number of turns per agent: {num_turns}\n")
    
    # Initialize agents
    agent_a = Agent(
        name="Debater A",
        role="proponent",
        description=f"argue in favor of the topic '{topic}'. Respond to Debater B's points.",
        model="gpt-4.1-nano"
    )
    
    agent_b = Agent(
         name="Debater B",
        role="opponent",
        description=f"argue against the topic '{topic}'. Respond to Debater A's points.",
        model="gpt-4.1-nano"
    )
    
    agents = [agent_a, agent_b]
    
    # Start the debate
    central_log = [] # A central place to log messages
    
    # Initial statement from Agent A
    print("--- Starting Debate ---")
    initial_query = f"Begin the debate. State your opening argument for the topic: {topic}"
    response_a = agent_a.generate_response(initial_query)
    central_log.append(agent_a.send_message(agent_b.name, response_a))
    print(f"{agent_a.name}: {response_a}\n")
    
    # Debate turns
    for i in range(num_turns * len(agents) - 1): # Total turns - 1 (first turn already done)
        current_agent_idx = (i + 1) % len(agents)
        current_agent = agents[current_agent_idx]
        previous_agent = agents[i % len(agents)]
        
        # In a real system, agents would pick up messages from a queue
        # Here, we manually pass the last message
        last_message = central_log[-1]
        if last_message['recipient'] == current_agent.name:
             current_agent.receive_message(last_message)
        
        response = current_agent.generate_response()
        
        # Determine recipient (the other agent)
        recipient_agent = agents[(current_agent_idx + 1) % len(agents)]
        central_log.append(current_agent.send_message(recipient_agent.name, response))
        print(f"{current_agent.name}: {response}\n")
        
        # Small delay to simulate thinking/processing time (optional)
        # time.sleep(1)

    print("--- Debate Concluded ---")

    # (Optional) A final agent could summarize the debate
    # For this example, we just print the log
    print("\n=== CENTRAL DEBATE LOG ===")
    for msg in central_log:
        print(f"[{msg['sender']} -> {msg['recipient']}]: {msg['content']}\n")
        
    return central_log

# Test the multi-agent pattern
debate_log = multi_agent_debate("whether AI will improve society", num_turns=2)


print("\nDone!")

=== MULTI-AGENT PATTERN EXAMPLE ===
Debate Topic: whether AI will improve society
Number of turns per agent: 2

--- Starting Debate ---
  Debater A is generating response...
  Debater A generated response.
Debater A: Thank you. I firmly believe that AI has the potential to significantly improve society across numerous dimensions. Firstly, AI can enhance healthcare by enabling early diagnosis, personalized treatment plans, and even predicting disease outbreaks, thereby saving lives and reducing costs. Secondly, AI-driven automation can increase productivity and efficiency in industries, freeing humans from monotonous tasks and allowing them to focus on more creative and strategic endeavors. Additionally, AI can facilitate better decision-making through data analysis, helping governments and organizations implement policies that are more effective and equitable. While there are challenges to address, the overall trajectory of AI development promises a future where society benefits from i

## Conclusion

This notebook provided practical examples of several common agentic design patterns using the OpenAI API. By combining these patterns, more sophisticated and autonomous AI systems can be built.

- **Prompt Chaining:** Sequential processing.
- **Routing/Handoff:** Directing tasks to specialized models/agents.
- **Parallelization:** Concurrent execution for efficiency.
- **Reflection:** Iterative self-improvement.
- **Tool Use:** Interacting with external environments.
- **Planning:** Decomposing complex goals into executable steps.
- **Multi-Agent:** Collaboration and interaction between independent agents.

Exploring combinations and variations of these patterns is key to building powerful agentic applications.